In [1]:
import os, time, importlib
import numpy as np
import cadquery as cq
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# -----------------------
# USER CONFIG
# -----------------------
TARGET_STEP = "tmp_target.step"  

# Interactive resolution (speed knobs)
NS = 50      # s samples (stations along blade)
NT = 120     # t samples around airfoil loop
HUB_NTH = 120
HUB_NZ  = 60

TARGET_TESS_TOL = 0.6   # larger = coarser target mesh = faster
UPDATE_DEBOUNCE_S = 0.03  # small debounce to avoid flooding updates

# -----------------------
# Read initial parameters from config_toroidal.py
# -----------------------
cfg = importlib.import_module("config_toroidal")

def params_from_config(cfg):
    # Flatten into scalar keys for sliders
    return dict(
        global_scale=float(cfg.global_scale),
        hub_radius=float(cfg.hub_radius),
        hub_length=float(cfg.hub_length),
        num_blades=int(cfg.num_blades),

        m=float(cfg.m),
        p=float(cfg.p),
        thickness=float(cfg.thickness),

        loc2_circ=float(cfg.loc_ctrl_point2[0]),
        loc2_z=float(cfg.loc_ctrl_point2[1]),
        loc2_rad=float(cfg.loc_ctrl_point2[2]),

        loc3_circ=float(cfg.loc_ctrl_point3[0]),
        loc3_z=float(cfg.loc_ctrl_point3[1]),
        loc3_rad=float(cfg.loc_ctrl_point3[2]),

        blade_circ=float(cfg.blade_vector[0]),
        blade_z=float(cfg.blade_vector[1]),

        a_AoA=float(cfg.a_AoA),
        b_AoA=float(cfg.b_AoA),
        c_AoA=float(cfg.c_AoA),
        d_AoA=float(cfg.d_AoA),
        e_AoA=float(cfg.e_AoA),

        a_scX=float(cfg.a_scX),
        b_scX=float(cfg.b_scX),
        c_scX=float(cfg.c_scX),
        d_scX=float(cfg.d_scX),
        e_scX=float(cfg.e_scX),

        a_scY=float(cfg.a_scY),
        b_scY=float(cfg.b_scY),
        c_scY=float(cfg.c_scY),
        d_scY=float(cfg.d_scY),
        e_scY=float(cfg.e_scY),

        apply_thickness_normal=bool(cfg.apply_thickness_normal),
    )

base = params_from_config(cfg)

# -----------------------
# Target STEP -> plotly mesh (once)
# -----------------------
def cq_shape_to_tris(shape, tol=0.6):
    verts, tris = shape.tessellate(tol)
    V = np.array([[v.x, v.y, v.z] for v in verts], dtype=np.float64)
    F = np.asarray(tris, dtype=np.int32)
    return V, F

target_shape = cq.importers.importStep(TARGET_STEP).val()
tV, tF = cq_shape_to_tris(target_shape, tol=TARGET_TESS_TOL)


# ============================================================
# FAST candidate mesh generation (NUMPY ONLY)
# ============================================================

def poly4(s, a,b,c,d,e):
    return a*s**4 + b*s**3 + c*s**2 + d*s + e

def clamp_scaling(s):
    # numeric version of your clamp
    return -1.0 + 1.0/(1.0 + np.exp(-200.0*(s - 0.01))) + 1.0/(1.0 + np.exp(-200.0*(0.99 - s)))

def naca_4digit_xy(tvals, m, p, thickness, apply_thickness_normal=False):
    """
    tvals in [0,2): upper for t<1 (x=t), lower for t>=1 (x=2-t)
    """
    tvals = np.asarray(tvals, dtype=np.float64)
    upper = tvals < 1.0
    x = np.where(upper, tvals, 2.0 - tvals)  # 0..1

    yt = 5.0 * thickness * (
        0.2969*np.sqrt(np.clip(x, 0, 1))
        - 0.1260*x
        - 0.3516*x**2
        + 0.2843*x**3
        - 0.1036*x**4
    )

    # camber
    yc = np.where(
        x <= p,
        (m / (p**2 + 1e-12)) * (2*p*x - x**2),
        (m / ((1-p)**2 + 1e-12)) * ((1 - 2*p) + 2*p*x - x**2)
    )

    if apply_thickness_normal:
        dyc_dx = np.where(
            x <= p,
            2*m/(p**2 + 1e-12) * (p - x),
            2*m/(((1-p)**2 + 1e-12)) * (p - x)
        )
        theta = np.arctan(dyc_dx)
    else:
        theta = 0.0

    yu = yc + yt*np.cos(theta)
    yl = yc - yt*np.cos(theta)
    xu = x - yt*np.sin(theta)
    xl = x + yt*np.sin(theta)

    y = np.where(upper, yu, yl)
    xout = np.where(upper, xu, xl) - 0.5
    return xout, y

def rational_cubic_bezier(P, w, s):
    """
    P: (4,3), w: (4,), s: (Ns,)
    """
    s = s[:, None]
    one = 1.0 - s
    B0 = one**3
    B1 = 3*s*one**2
    B2 = 3*s**2*one
    B3 = s**3

    W0 = B0 * w[0]
    W1 = B1 * w[1]
    W2 = B2 * w[2]
    W3 = B3 * w[3]

    denom = (W0 + W1 + W2 + W3)
    num = W0 * P[0] + W1 * P[1] + W2 * P[2] + W3 * P[3]
    return num / denom

def frenet_frames(C):
    """
    numeric Frenet-like frame (T,N,B) with re-orthogonalization
    """
    dC = np.gradient(C, axis=0)
    T = dC / (np.linalg.norm(dC, axis=1, keepdims=True) + 1e-12)

    dT = np.gradient(T, axis=0)
    N = dT / (np.linalg.norm(dT, axis=1, keepdims=True) + 1e-12)

    B = np.cross(T, N)
    B = B / (np.linalg.norm(B, axis=1, keepdims=True) + 1e-12)

    # re-orthogonalize N
    N = np.cross(B, T)
    N = N / (np.linalg.norm(N, axis=1, keepdims=True) + 1e-12)
    return T, N, B

def build_candidate_mesh_arrays(params, ns=NS, nt=NT, hub_nth=HUB_NTH, hub_nz=HUB_NZ):
    """
    Returns (V, F) triangles for candidate = hub + blades.
    No CAD, no files. Designed for fast interactive updates.
    """
    # unpack
    gs = float(params["global_scale"])
    hub_r = float(params["hub_radius"])
    hub_L = float(params["hub_length"])
    nb = int(params["num_blades"])

    m = float(params["m"])
    p = float(params["p"])
    thick = float(params["thickness"])
    apply_thick_normal = bool(params["apply_thickness_normal"])

    loc2 = [float(params["loc2_circ"]), float(params["loc2_z"]), float(params["loc2_rad"])]
    loc3 = [float(params["loc3_circ"]), float(params["loc3_z"]), float(params["loc3_rad"])]
    bv   = [float(params["blade_circ"]), float(params["blade_z"])]

    a_AoA,b_AoA,c_AoA,d_AoA,e_AoA = [float(params[k]) for k in ["a_AoA","b_AoA","c_AoA","d_AoA","e_AoA"]]
    a_scX,b_scX,c_scX,d_scX,e_scX = [float(params[k]) for k in ["a_scX","b_scX","c_scX","d_scX","e_scX"]]
    a_scY,b_scY,c_scY,d_scY,e_scY = [float(params[k]) for k in ["a_scY","b_scY","c_scY","d_scY","e_scY"]]

    # sampling
    s = np.linspace(0.0, 1.0, ns, endpoint=True)
    t = np.linspace(0.0, 2.0, nt, endpoint=False)

    # 2D airfoil
    x2d, y2d = naca_4digit_xy(t, m=m, p=p, thickness=thick, apply_thickness_normal=apply_thick_normal)

    # along-s AoA and scaling
    AoA = poly4(s, a_AoA,b_AoA,c_AoA,d_AoA,e_AoA)
    scx = poly4(s, a_scX,b_scX,c_scX,d_scX,e_scX)
    scy = poly4(s, a_scY,b_scY,c_scY,d_scY,e_scY)
    cl = clamp_scaling(s)
    scx = scx * cl
    scy = scy * cl

    ca = np.cos(AoA)[:, None]
    sa = np.sin(AoA)[:, None]

    # rotated+scaled airfoil in local (N,B) plane
    Xr = (ca*x2d[None,:] - sa*y2d[None,:]) * scx[:,None]
    Yr = (sa*x2d[None,:] + ca*y2d[None,:]) * scy[:,None]

    # centerline control points (with the “relative z0 fix”)
    inset_ratio = 4.0/8.0
    blade_hub_r = inset_ratio * hub_r
    z0 = hub_L/2.0 - 1.0

    theta4 = bv[0] / (blade_hub_r + 1e-12)  # arc/r
    theta2 = loc2[0] / (blade_hub_r + 1e-12)
    theta3 = loc3[0] / (blade_hub_r + 1e-12)

    P1 = np.array([blade_hub_r, 0.0, z0], dtype=np.float64)
    P4 = np.array([blade_hub_r*np.cos(theta4), blade_hub_r*np.sin(theta4), z0 - bv[1]], dtype=np.float64)
    r2 = blade_hub_r + loc2[2]
    r3 = blade_hub_r + loc3[2]
    P2 = np.array([r2*np.cos(theta2), r2*np.sin(theta2), z0 - loc2[1]], dtype=np.float64)
    P3 = np.array([r3*np.cos(theta3), r3*np.sin(theta3), z0 - loc3[1]], dtype=np.float64)

    P = np.stack([P1,P2,P3,P4], axis=0)
    w = np.array([1.0,1.0,1.0,1.0], dtype=np.float64)

    C = rational_cubic_bezier(P, w, s)   # (ns,3)
    _, N, B = frenet_frames(C)           # (ns,3)

    # blade surface points (ns, nt, 3)
    blade = C[:,None,:] + Xr[:,:,None]*N[:,None,:] + Yr[:,:,None]*B[:,None,:]

    # Triangulate structured grid for one blade
    # vertex indexing: idx = si*nt + ti
    V0 = blade.reshape(-1,3)
    faces = []
    for si in range(ns-1):
        row0 = si*nt
        row1 = (si+1)*nt
        for ti in range(nt):
            tj = (ti+1) % nt  # wrap around
            a = row0 + ti
            b = row0 + tj
            c = row1 + ti
            d = row1 + tj
            faces.append([a, c, b])
            faces.append([b, c, d])
    F0 = np.array(faces, dtype=np.int32)

    # replicate blades
    allV = []
    allF = []
    v_offset = 0
    for bi in range(nb):
        ang = 2*np.pi*bi/nb
        ca_, sa_ = np.cos(ang), np.sin(ang)
        R = np.array([[ca_, -sa_, 0.0],
                      [sa_,  ca_, 0.0],
                      [0.0,  0.0, 1.0]], dtype=np.float64)
        Vb = V0 @ R.T
        allV.append(Vb)
        allF.append(F0 + v_offset)
        v_offset += Vb.shape[0]

    Vb_all = np.vstack(allV)
    Fb_all = np.vstack(allF)

    # hub cylinder mesh
    th = np.linspace(0, 2*np.pi, hub_nth, endpoint=False)
    zz = np.linspace(-hub_L/2.0, hub_L/2.0, hub_nz)
    TH, ZZ = np.meshgrid(th, zz, indexing="xy")

    Xh = hub_r*np.cos(TH)
    Yh = hub_r*np.sin(TH)
    Zh = ZZ
    Vh = np.column_stack([Xh.ravel(), Yh.ravel(), Zh.ravel()]).astype(np.float64)

    faces_h = []
    for zi in range(hub_nz-1):
        r0 = zi*hub_nth
        r1 = (zi+1)*hub_nth
        for ti in range(hub_nth):
            tj = (ti+1) % hub_nth
            a = r0 + ti
            b = r0 + tj
            c = r1 + ti
            d = r1 + tj
            faces_h.append([a, c, b])
            faces_h.append([b, c, d])
    Fh = np.array(faces_h, dtype=np.int32)

    # combine hub + blades
    V = np.vstack([Vh, Vb_all])
    F = np.vstack([Fh, Fb_all + Vh.shape[0]])

    # global scale
    V = V * gs
    return V, F


# -----------------------
# Plotly FigureWidget
# -----------------------
def make_mesh_trace(V,F, name, color, opacity):
    return go.Mesh3d(
        x=V[:,0], y=V[:,1], z=V[:,2],
        i=F[:,0], j=F[:,1], k=F[:,2],
        name=name, color=color, opacity=opacity,
        flatshading=True, showscale=False
    )

candV, candF = build_candidate_mesh_arrays(base)

fig = go.FigureWidget()
fig.add_trace(make_mesh_trace(tV, tF, "Target", "red", 0.22))
fig.add_trace(make_mesh_trace(candV, candF, "Parametric", "blue", 0.35))

fig.update_layout(
    title="Overlay: Target (red) vs Parametric (blue) — fast NumPy mesh updates",
    scene=dict(aspectmode="data"),
    margin=dict(l=0, r=0, t=40, b=0),
    height=720
)
display(fig)


# ============================================================
# Sliders
# ============================================================

def fs(desc, value, minv, maxv, step, fmt=".4f"):
    return widgets.FloatSlider(
        value=value, min=minv, max=maxv, step=step,
        description=desc, continuous_update=True,
        readout_format=fmt, layout=widgets.Layout(width="420px")
    )

def islider(desc, value, minv, maxv, step=1):
    return widgets.IntSlider(
        value=value, min=minv, max=maxv, step=step,
        description=desc, continuous_update=True,
        layout=widgets.Layout(width="420px")
    )

def cb(desc, value):
    return widgets.Checkbox(value=value, description=desc)

# Core
w_global_scale = fs("global_scale", base["global_scale"], 1.0, 20.0, 0.05, ".3f")
w_hub_radius   = fs("hub_radius",   base["hub_radius"],   0.5, 20.0, 0.05, ".3f")
w_hub_length   = fs("hub_length",   base["hub_length"],   1.0, 80.0, 0.1,  ".2f")
w_num_blades   = islider("num_blades", base["num_blades"], 1, 8, 1)

# Airfoil
w_m = fs("m", base["m"], 0.0, 0.2, 0.002, ".4f")
w_p = fs("p", base["p"], 0.05, 0.95, 0.005, ".3f")
w_thickness = fs("thickness", base["thickness"], 0.05, 2.5, 0.01, ".3f")
w_apply = cb("apply_thickness_normal", base["apply_thickness_normal"])

# Centerline
w_loc2_circ = fs("loc2_circ", base["loc2_circ"], -80, 80, 0.25, ".2f")
w_loc2_z    = fs("loc2_z",    base["loc2_z"],    -80, 80, 0.25, ".2f")
w_loc2_rad  = fs("loc2_rad",  base["loc2_rad"],   0, 120, 0.25, ".2f")

w_loc3_circ = fs("loc3_circ", base["loc3_circ"], -80, 80, 0.25, ".2f")
w_loc3_z    = fs("loc3_z",    base["loc3_z"],    -80, 80, 0.25, ".2f")
w_loc3_rad  = fs("loc3_rad",  base["loc3_rad"],   0, 120, 0.25, ".2f")

w_blade_circ = fs("blade_circ", base["blade_circ"], -150, 150, 0.25, ".2f")
w_blade_z    = fs("blade_z",    base["blade_z"],    -150, 150, 0.25, ".2f")

# AoA
w_aAoA = fs("a_AoA", base["a_AoA"], -10*np.pi, 10*np.pi, 0.05, ".3f")
w_bAoA = fs("b_AoA", base["b_AoA"], -10*np.pi, 10*np.pi, 0.05, ".3f")
w_cAoA = fs("c_AoA", base["c_AoA"], -10*np.pi, 10*np.pi, 0.05, ".3f")
w_dAoA = fs("d_AoA", base["d_AoA"], -10*np.pi, 10*np.pi, 0.05, ".3f")
w_eAoA = fs("e_AoA", base["e_AoA"], -10*np.pi, 10*np.pi, 0.05, ".3f")

# Scale polynomials
w_a_scX = fs("a_scX", base["a_scX"], -10, 10, 0.05, ".3f")
w_b_scX = fs("b_scX", base["b_scX"], -10, 10, 0.05, ".3f")
w_c_scX = fs("c_scX", base["c_scX"], -10, 10, 0.05, ".3f")
w_d_scX = fs("d_scX", base["d_scX"], -10, 10, 0.05, ".3f")
w_e_scX = fs("e_scX", base["e_scX"],  0.01, 10, 0.05, ".3f")

w_a_scY = fs("a_scY", base["a_scY"], -10, 10, 0.05, ".3f")
w_b_scY = fs("b_scY", base["b_scY"], -10, 10, 0.05, ".3f")
w_c_scY = fs("c_scY", base["c_scY"], -10, 10, 0.05, ".3f")
w_d_scY = fs("d_scY", base["d_scY"], -10, 10, 0.05, ".3f")
w_e_scY = fs("e_scY", base["e_scY"],  0.01, 10, 0.05, ".3f")

# Layout
group0 = widgets.VBox([w_global_scale, w_hub_radius, w_hub_length, w_num_blades])
group1 = widgets.VBox([w_m, w_p, w_thickness, w_apply])
group2 = widgets.VBox([
    widgets.HTML("<b>loc_ctrl_point2</b>"), w_loc2_circ, w_loc2_z, w_loc2_rad,
    widgets.HTML("<b>loc_ctrl_point3</b>"), w_loc3_circ, w_loc3_z, w_loc3_rad,
    widgets.HTML("<b>blade_vector</b>"), w_blade_circ, w_blade_z
])
group3 = widgets.VBox([w_aAoA, w_bAoA, w_cAoA, w_dAoA, w_eAoA])
group4 = widgets.VBox([w_a_scX, w_b_scX, w_c_scX, w_d_scX, w_e_scX])
group5 = widgets.VBox([w_a_scY, w_b_scY, w_c_scY, w_d_scY, w_e_scY])

acc = widgets.Accordion(children=[group0, group1, group2, group3, group4, group5])
acc.set_title(0, "Hub / Global")
acc.set_title(1, "Airfoil")
acc.set_title(2, "Centerline")
acc.set_title(3, "AoA poly")
acc.set_title(4, "Scale X poly")
acc.set_title(5, "Scale Y poly")
display(acc)


# ============================================================
# Live update (fast)
# ============================================================

_last = 0.0
_pending = False

def read_params():
    return dict(
        global_scale=w_global_scale.value,
        hub_radius=w_hub_radius.value,
        hub_length=w_hub_length.value,
        num_blades=w_num_blades.value,

        m=w_m.value,
        p=w_p.value,
        thickness=w_thickness.value,
        apply_thickness_normal=w_apply.value,

        loc2_circ=w_loc2_circ.value,
        loc2_z=w_loc2_z.value,
        loc2_rad=w_loc2_rad.value,

        loc3_circ=w_loc3_circ.value,
        loc3_z=w_loc3_z.value,
        loc3_rad=w_loc3_rad.value,

        blade_circ=w_blade_circ.value,
        blade_z=w_blade_z.value,

        a_AoA=w_aAoA.value, b_AoA=w_bAoA.value, c_AoA=w_cAoA.value, d_AoA=w_dAoA.value, e_AoA=w_eAoA.value,
        a_scX=w_a_scX.value, b_scX=w_b_scX.value, c_scX=w_c_scX.value, d_scX=w_d_scX.value, e_scX=w_e_scX.value,
        a_scY=w_a_scY.value, b_scY=w_b_scY.value, c_scY=w_c_scY.value, d_scY=w_d_scY.value, e_scY=w_e_scY.value,
    )

def update(_=None):
    global _last, _pending
    now = time.time()
    if now - _last < UPDATE_DEBOUNCE_S:
        _pending = True
        return
    _last = now
    _pending = False

    p = read_params()
    try:
        V,F = build_candidate_mesh_arrays(p)
    except Exception as e:
        # if it errors, just ignore the update
        # (common if sliders go extreme)
        return

    with fig.batch_update():
        # candidate is trace index 1
        fig.data[1].x = V[:,0]
        fig.data[1].y = V[:,1]
        fig.data[1].z = V[:,2]
        fig.data[1].i = F[:,0]
        fig.data[1].j = F[:,1]
        fig.data[1].k = F[:,2]

    if _pending:
        update()

controls = [
    w_global_scale, w_hub_radius, w_hub_length, w_num_blades,
    w_m, w_p, w_thickness, w_apply,
    w_loc2_circ, w_loc2_z, w_loc2_rad,
    w_loc3_circ, w_loc3_z, w_loc3_rad,
    w_blade_circ, w_blade_z,
    w_aAoA, w_bAoA, w_cAoA, w_dAoA, w_eAoA,
    w_a_scX, w_b_scX, w_c_scX, w_d_scX, w_e_scX,
    w_a_scY, w_b_scY, w_c_scY, w_d_scY, w_e_scY
]

for c in controls:
    c.observe(update, names="value")

print("Fast interactive overlay ready — move sliders and the blue mesh should update immediately.")


FigureWidget({
    'data': [{'color': 'red',
              'flatshading': True,
              'i': {'bdata': ('aAAAAGgAAABpAAAAaQAAAGoAAABqAA' ... 'DFKQIAxSkCAMUpAgDFKQIAxSkCAA=='),
                    'dtype': 'i4'},
              'j': {'bdata': ('AgAAAAMAAAAEAAAAAwAAAAUAAAAEAA' ... 'DMKQIAtSkCAMIpAgDGKQIAyykCAA=='),
                    'dtype': 'i4'},
              'k': {'bdata': ('ZwAAAAIAAAADAAAAaAAAAAQAAABpAA' ... 'DZKQIAwikCAMQpAgDLKQIAzCkCAA=='),
                    'dtype': 'i4'},
              'name': 'Target',
              'opacity': 0.22,
              'showscale': False,
              'type': 'mesh3d',
              'uid': '363e1eb9-84d5-4111-b206-b1f0a556cf42',
              'x': {'bdata': ('t7qbTZP6PMCNr8zaEsc8wNB/bsxE0z' ... 'iXQj5A0MRHgylzPkCcAIlmWpA+QA=='),
                    'dtype': 'f8'},
              'y': {'bdata': ('vp9MI/iqI0BKBeXm3tAkQDiItMjH5D' ... 'lxPhJAeYoWYTBgCEDBDJ2n9Gf4Pw=='),
                    'dtype': 'f8'},
              'z': {'bdata': ('8uMxFrDxT

Accordion(children=(VBox(children=(FloatSlider(value=7.5, description='global_scale', layout=Layout(width='420…

Fast interactive overlay ready — move sliders and the blue mesh should update immediately.


In [ ]:
import datetime
import cadquery as cq

# --- choose output filenames ---
stamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
out_step = f"prop_manual_fit_{stamp}.step"
out_stl  = f"prop_manual_fit_{stamp}.stl"

# --- grab current params from sliders ---
params_now = read_params()

print("Exporting with current slider parameters:")
for k in sorted(params_now.keys()):
    print(f"  {k}: {params_now[k]}")

# ============================
# (A) High-quality STEP export (CadQuery loft)
# ============================

# You can turn these up for final quality:
FINAL_S_RES = 25
FINAL_T_RES = 60

print("\nBuilding CadQuery solid (this may take a bit)...")
solid = build_propeller(
    params_now,
    s_resolution_cad=FINAL_S_RES,
    t_resolution_cad=FINAL_T_RES,
    verbose=True
)

print("Exporting STEP...")
cq.exporters.export(solid, out_step, "STEP")
print("Wrote:", out_step)


Exporting with current slider parameters:
  a_AoA: -0.01592653589793258
  a_scX: 1.0
  a_scY: 0.0
  apply_thickness_normal: False
  b_AoA: -0.01592653589793258
  b_scX: 0.0
  b_scY: 0.0
  blade_circ: -12.500000000000028
  blade_z: 16.25
  c_AoA: -0.01592653589793258
  c_scX: -2.0
  c_scY: -1.0
  d_AoA: 3.134073464102066
  d_scX: 6.100000000000001
  d_scY: 0.0
  e_AoA: -0.01592653589793258
  e_scX: 5.710000000000001
  e_scY: 2.01
  global_scale: 7.5
  hub_length: 18.0
  hub_radius: 4.999999999999999
  loc2_circ: -0.25
  loc2_rad: 35.50000000000001
  loc2_z: 14.0
  loc3_circ: -3.75
  loc3_rad: 21.75
  loc3_z: 7.75
  m: 0.12
  num_blades: 3
  p: 0.54
  thickness: 0.75

Building CadQuery solid (this may take a bit)...
Exporting STEP...
Wrote: prop_manual_fit_20260211_144411.step
Wrote: prop_manual_fit_20260211_144411.stl


C:\Users\varun\AppData\Local\Temp\ipykernel_2744\1696361742.py:47: DeprecationWarning:

`remove_duplicate_faces` is deprecated and will be removed in March 2024: replace with `mesh.update_faces(mesh.unique_faces())`

C:\Users\varun\AppData\Local\Temp\ipykernel_2744\1696361742.py:48: DeprecationWarning:

`remove_degenerate_faces` is deprecated and will be removed in March 2024 replace with `self.update_faces(self.nondegenerate_faces(height=height))`

